In [1]:
import requests
import time
import re
import pandas as pd
import numpy as np
import random
from bs4 import BeautifulSoup

In [2]:
projects=pd.DataFrame(columns=['kickstarter_page','funding', 'campaign_dates', 'backers', 'category'])

In [3]:
#kicktraq archive page numbers 1-8000 are available
page_numbers = np.random.randint(1,high=2000,size=200)
for page_num in page_numbers:
    url = 'https://www.kicktraq.com/archive/?page=' + str(page_num)
    filename = 'kicktraq_files/kicktraq_page_num' + str(page_num)
    print('url is ' + url)
    kickstarter_page = []
    funding = []
    campaign_dates = []
    backers = []
    categories = []
    
    response = requests.get(url)
    time.sleep(3)
    
    for i in range (1,10):
        if response.status_code == 200:
            break
        else:
            print('response code: '+str(response.status_code))
            time.sleep(10)
            response = requests.get(url)
        
    
    page = response.text
    soup = BeautifulSoup(page,'lxml')
    
    
    all_projects_on_page = soup.find_all(class_="project first odd closed")
    all_projects_on_page = all_projects_on_page + soup.find_all(class_="project odd closed")
    all_projects_on_page = all_projects_on_page + soup.find_all(class_="project closed")

    for project in all_projects_on_page: 
        #if statement tests to see if funding info is available. If it is, we include the project. If not, we don't!
        funding_info = project.find(class_='project-details').find(text=re.compile('Funding: .\d*.\d* of .\d*.\d*'))
        if funding_info != None:
            funding.append(str(funding_info).replace('$',''))

            kickstarter_page.append(str(project.find('h2').find('a')['href']))
            campaign_dates.append(str(project.find(class_='project-details').find(text=re.compile('Dates: \w* \d*\w* \-\> \w* \d*\w* \(\d*\)'))))
            backers.append(str(project.find(class_='project-details').find(text=re.compile('Backers: \d*'))))
            categories.append(str(project.find(class_='project-cat').find('a').text))
            d = {'kickstarter_page': kickstarter_page,'funding': funding,
                 'campaign_dates': campaign_dates, 'backers': backers,
                 'category': categories}
            page_df = pd.DataFrame(d)
    
    page_df.to_csv(filename,index=False, encoding='utf-8')
    
    projects = pd.concat([projects, page_df])
    
    
    
    time.sleep(10)


url is https://www.kicktraq.com/archive/?page=1837
url is https://www.kicktraq.com/archive/?page=634
url is https://www.kicktraq.com/archive/?page=191
url is https://www.kicktraq.com/archive/?page=1154
url is https://www.kicktraq.com/archive/?page=1487
response code: 404
url is https://www.kicktraq.com/archive/?page=644
response code: 404
url is https://www.kicktraq.com/archive/?page=1534
url is https://www.kicktraq.com/archive/?page=751
url is https://www.kicktraq.com/archive/?page=438
response code: 404
url is https://www.kicktraq.com/archive/?page=1955
response code: 404
response code: 404
url is https://www.kicktraq.com/archive/?page=978
response code: 404
url is https://www.kicktraq.com/archive/?page=1117
url is https://www.kicktraq.com/archive/?page=176
response code: 404
url is https://www.kicktraq.com/archive/?page=998
url is https://www.kicktraq.com/archive/?page=1237
url is https://www.kicktraq.com/archive/?page=1892
url is https://www.kicktraq.com/archive/?page=1895
url is h

url is https://www.kicktraq.com/archive/?page=206
response code: 404
url is https://www.kicktraq.com/archive/?page=77
response code: 404
url is https://www.kicktraq.com/archive/?page=1635
url is https://www.kicktraq.com/archive/?page=82
url is https://www.kicktraq.com/archive/?page=1976
response code: 404
url is https://www.kicktraq.com/archive/?page=1173
url is https://www.kicktraq.com/archive/?page=1617
url is https://www.kicktraq.com/archive/?page=1755
response code: 404
url is https://www.kicktraq.com/archive/?page=1222
url is https://www.kicktraq.com/archive/?page=1885
url is https://www.kicktraq.com/archive/?page=812
response code: 404
response code: 404
url is https://www.kicktraq.com/archive/?page=1640
response code: 404
url is https://www.kicktraq.com/archive/?page=1254
url is https://www.kicktraq.com/archive/?page=85
response code: 404
response code: 404
url is https://www.kicktraq.com/archive/?page=1477
response code: 404
response code: 404
url is https://www.kicktraq.com/ar

In [4]:
import pickle
with open('page_numbers.pickle', 'wb') as f:
    pickle.dump(page_numbers,f)
print(page_numbers)

[1837  634  191 1154 1487  644 1534  751  438 1955  978 1117  176  998
 1237 1892 1895 1211 1265 1739  397 1083 1443 1934  907  303 1612  793
 1125  992  574  922 1928  889  104  326  922 1393 1100 1095 1800 1771
  690  488 1104 1961  298 1679 1555  530 1646 1461 1397 1290  579 1829
  416 1570 1763  539  719  512 1610  918 1785 1575   42  133 1721 1512
 1752  887  826  533  279 1129 1834 1005  532  212  636 1889 1365  653
 1083  791 1477 1330 1226 1119  865 1587 1710  415 1920  550 1211 1396
 1313 1208  125  946  455 1318 1386  970   93 1741 1131  922 1008  277
 1860  391  602  578 1314  745 1548 1560  286  255 1322   96 1556 1536
  307  998 1892  575 1667 1970 1713 1039  169  206   77 1635   82 1976
 1173 1617 1755 1222 1885  812 1640 1254   85 1477 1894 1140  923 1281
  998 1872   45 1436  231 1844 1386  410  496 1948 1350  950 1395 1181
 1414   14  495  142 1202 1431 1181 1648 1538 1510  457  571 1431  224
 1565  694  763  453  892 1868 1365 1545 1683 1952  550 1661 1517  161
  456 

In [32]:
import pickle
with open('kickstarter_data_raw.pickle', 'rb') as f:
    projects = pd.DataFrame(pickle.load(f))
projects.head()

,kickstarter_page,funding,campaign_dates,backers,category,description_length,n_pledges,pledge_level_min,pledge_level_max,pledge_level_stddev,currency
0,/projects/1749739352/help-nola-aromatherapie-g...,"\n\t\t\t\t\tFunding: 36 of 1,500 (",\n\t\t\t\t\tCampaign Dates: July 14th -> Augus...,\n\t\t\t\t\tBackers: 3,Crafts,0,0,0,0,0,$
1,/projects/543502313/my-post-apocalyptic-family/,"\n\t\t\t\t\tFunding: 0 of 1,200 (",\n\t\t\t\t\tCampaign Dates: July 14th -> Augus...,\n\t\t\t\t\tBackers: 0,Photography,0,0,0,0,0,$
2,/projects/1519505761/oklahoma-the-way-i-see-it...,"\n\t\t\t\t\tFunding: 7,433 of 12,500 (",\n\t\t\t\t\tCampaign Dates: July 14th -> Augus...,\n\t\t\t\t\tBackers: 103,Photography,0,0,0,0,0,$
3,/projects/1832560047/a-youtube-love-story-help...,"\n\t\t\t\t\tFunding: 217 of 1,500 (",\n\t\t\t\t\tCampaign Dates: July 9th -> August...,\n\t\t\t\t\tBackers: 9,Film & Video,0,0,0,0,0,$
4,/projects/1569100255/storybook-friends-come-in...,"\n\t\t\t\t\tFunding: 65 of 1,500 (",\n\t\t\t\t\tCampaign Dates: July 14th -> Augus...,\n\t\t\t\t\tBackers: 3,Publishing,0,0,0,0,0,$


In [33]:
print(projects.kickstarter_page.unique().shape)
print(projects.duplicated(subset='kickstarter_page'))

(1995,)
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
      ...  
14    False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
Length: 1995, dtype: bool


In [34]:
projects.drop_duplicates(subset='kickstarter_page', inplace=True)

In [35]:
projects['description_length'] = 0
projects['n_pledges'] = 0
projects['pledge_level_min'] = 0
projects['pledge_level_max'] = 0
projects['pledge_level_stddev'] = 0
projects['currency'] = '$'

In [36]:
def get_kickstarter_data(proj_series):
    url = 'https://www.kickstarter.com/' + proj_series.kickstarter_page
    print(url)    
    response = requests.get(url)
    for i in range (1,100):
        if response.status_code == 200:
            break
        else:
            time.sleep(5)
            response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,'lxml')

    pledges = soup.find_all(class_='pledge__info')
    
    n_pledges = len(pledges)
    
    pledge_amounts=[]
    for pledge in pledges:
        pledge_amount = str(pledge.find(class_='money').text.replace('$',''))
        currency = ''
        #while the first character in pledge_amount is not an integer
        while not (re.match('\d',pledge_amount[0])):
            currency = currency + (pledge_amount[0])
            pledge_amount = pledge_amount[1:]
        if currency == '':
            currency = '$'
        proj_series['currency'] = currency
        pledge_amounts.append(int(pledge_amount.replace(',','')))

    try:
        descript_paragraphs = soup.find(class_="full-description js-full-description responsive-media formatted-lists").find_all('p')
        description_words = 0
        for par in descript_paragraphs:
            description_words += len(str(par.text).split())
        
        proj_series.description_length = description_words
        proj_series.n_pledges = n_pledges
        proj_series.pledge_level_min = min(pledge_amounts)
        proj_series.pledge_level_max = max(pledge_amounts)
        proj_series.pledge_level_stddev = np.std(pledge_amounts)
    except:
        pass
        
    return proj_series
        
    time.sleep(2+random.uniform(0,2))

In [37]:
projects = projects.apply(get_kickstarter_data, axis=1)

https://www.kickstarter.com//projects/1749739352/help-nola-aromatherapie-get-started/
https://www.kickstarter.com//projects/1749739352/help-nola-aromatherapie-get-started/
https://www.kickstarter.com//projects/543502313/my-post-apocalyptic-family/
https://www.kickstarter.com//projects/1519505761/oklahoma-the-way-i-see-it-the-book/
https://www.kickstarter.com//projects/1832560047/a-youtube-love-story-help-fund-the-big-move/
https://www.kickstarter.com//projects/1569100255/storybook-friends-come-in-all-sizes-the-monk-and-t/
https://www.kickstarter.com//projects/1932603621/road-kill-photo-bomb/
https://www.kickstarter.com//projects/784815854/freakingsweetjars/
https://www.kickstarter.com//projects/1501427760/merican-pride-monster-truck-t-shirt/
https://www.kickstarter.com//projects/799850493/strapshot-ev1-for-mirrorless-compact-and-four-thir/
https://www.kickstarter.com//projects/881878832/community-garden-project/
https://www.kickstarter.com//projects/4379010/one-little-word-my-journey-t

https://www.kickstarter.com//projects/duadikos/moriartys-machinations/
https://www.kickstarter.com//projects/600963220/apistogramma-dwarf-cichlids-database/
https://www.kickstarter.com//projects/709754364/venexiana-dark-playing-cards/
https://www.kickstarter.com//projects/196686747/magical-cubes-of-christmas-wonderment/
https://www.kickstarter.com//projects/80975289/home-made-brittles-start-up-fund-raising/
https://www.kickstarter.com//projects/marcgunn/celtic-christmas-music-greeting-cards/
https://www.kickstarter.com//projects/200746019/urban-hoppers-farm-for-truely-humanely-raised-rabb/
https://www.kickstarter.com//projects/playfulnatureofmen/the-playful-nature-of-men-the-first-photograpy-boo/
https://www.kickstarter.com//projects/troopercooper/drone-print-issues-for-our-sci-fi-action-series/
https://www.kickstarter.com//projects/theedge/i-have-a-blessed-life-a-memoir/
https://www.kickstarter.com//projects/32397045/finding-stan-musial/
https://www.kickstarter.com//projects/673108581

https://www.kickstarter.com//projects/1816693923/la-femme-flambeau-short-film/
https://www.kickstarter.com//projects/2137748986/hallyu-wave/
https://www.kickstarter.com//projects/379276366/reanna-silvermen-sexy-cooking-clips/
https://www.kickstarter.com//projects/nighthawks/control-alt-delete/
https://www.kickstarter.com//projects/195166246/the-life-size-mousetrap-bringing-physics-to-life/
https://www.kickstarter.com//projects/2099746053/stop-talking-and-do-it/
https://www.kickstarter.com//projects/1550139987/hd-designs-t-shirts/
https://www.kickstarter.com//projects/qdc/international-childrens-book-release-project/
https://www.kickstarter.com//projects/595133389/quintessence-the-game/
https://www.kickstarter.com//projects/1959387746/leave-those-damn-drugs-alone-stage-play/
https://www.kickstarter.com//projects/bnkbgames/nightlight-a-cooperative-card-game/
https://www.kickstarter.com//projects/criminalbirds/criminal-birds-ep-release-and-tour-needs/
https://www.kickstarter.com//projects

https://www.kickstarter.com//projects/664475853/bluebird-weather-short-film/
https://www.kickstarter.com//projects/2110623383/my-status-bracelet/
https://www.kickstarter.com//projects/1053966614/lux-by-tory/
https://www.kickstarter.com//projects/jefftrier/shadow-hunter-expanded/
https://www.kickstarter.com//projects/1074842080/nirmal-for-disruptive-eating/
https://www.kickstarter.com//projects/brittanygoss/writers-residency-at-the-vermont-studio-center/
https://www.kickstarter.com//projects/2137393854/ysleep-truckies-change-gears-steer-clear-sleep-wel/
https://www.kickstarter.com//projects/1327285887/vegan-cupcakes-by-toadie/
https://www.kickstarter.com//projects/galauner-justis/stalwart-pencils-industrial-strength-craftmanship/
https://www.kickstarter.com//projects/412908306/i-hate-erasers-o/
https://www.kickstarter.com//projects/875932575/keep-it-safe/
https://www.kickstarter.com//projects/2130730794/bees-knees-hosiery-express-yourself-from-the-waist/
https://www.kickstarter.com//pro

https://www.kickstarter.com//projects/263581559/this-is-a-story-that-needs-to-be-told-raped-financ/
https://www.kickstarter.com//projects/748554295/the-golf-ball-seeker-find-underwater-golf-balls/
https://www.kickstarter.com//projects/13317007/619-brewings-debut-batch/
https://www.kickstarter.com//projects/354113969/sanar/
https://www.kickstarter.com//projects/1974115405/streamage/
https://www.kickstarter.com//projects/2083741358/eagle-project-portable-modular-stage-construction/
https://www.kickstarter.com//projects/1813412892/brewmasters-kitchen-all-american-beer-micro-brew-a/
https://www.kickstarter.com//projects/1467974238/help-us-fund-a-new-hobby-shop-for-everyone/
https://www.kickstarter.com//projects/902842455/sunshine-and-color-collide/
https://www.kickstarter.com//projects/1578425651/get-this-day-burns-next-album-before-anyone-else/
https://www.kickstarter.com//projects/1213297354/saturated-fat-first-neutral-then-deadly-now-uncert/
https://www.kickstarter.com//projects/2018643

...

https://www.kickstarter.com//projects/1037403017/monney-tools-young-adults-guide-to-financial-freed/
https://www.kickstarter.com//projects/2006539107/maddy-moos-ice-cream-cart-by-10-year-old-maddy-bar/
https://www.kickstarter.com//projects/1044352266/giving-everyone-more-than-food-i-am-creating-an-ex/
https://www.kickstarter.com//projects/1058751300/drift-0/
https://www.kickstarter.com//projects/625315493/the-most-addictive-bedtime-stories-in-history/
https://www.kickstarter.com//projects/1661575903/tennessee-williams-one-arm-minnesota-fringe/
https://www.kickstarter.com//projects/1685836827/lucky-13-saloon-live/
https://www.kickstarter.com//projects/330548892/crusin-beauties-2015-calendar-0/
https://www.kickstarter.com//projects/1214062338/mark-hayes-requiem-recording/
https://www.kickstarter.com//projects/1888261595/twin-cities-tees-with-a-story/
https://www.kickstarter.com//projects/2134752613/the-greatest-birdhouse-youll-ever-see/
https://www.kickstarter.com//projects/1350470558/sh

https://www.kickstarter.com//projects/harrywang/thinkpower-battery-pack-10000mah-recharges-in-90-m/
https://www.kickstarter.com//projects/514611806/suitable-playing-cards/
https://www.kickstarter.com//projects/photosbysutter/the-world-is-a-beautiful-place-and-i-want-to-share/
https://www.kickstarter.com//projects/62739458/beyond-environment/
https://www.kickstarter.com//projects/429166063/makeing-a-game-server/
https://www.kickstarter.com//projects/442187461/wonderful-photos-of-cincinnati/
https://www.kickstarter.com//projects/815926680/tiny-little-specks/
https://www.kickstarter.com//projects/425171117/needle-point-mezuzahs/
https://www.kickstarter.com//projects/228664104/laughing-for-hope-in-oso-wa/
https://www.kickstarter.com//projects/1084007386/ellie-and-starlight/
https://www.kickstarter.com//projects/1013148356/three-childrens-books-about-two-dogs-harry-and-mat/
https://www.kickstarter.com//projects/1334680716/help-get-ellie-grace-couture-funded/
https://www.kickstarter.com//pro

https://www.kickstarter.com//projects/928909043/upgrade-to-our-laser-engraving-business-need-a-big/
https://www.kickstarter.com//projects/352515068/azauth-an-opensource-oauth2-server/
https://www.kickstarter.com//projects/2113750962/the-ellen-degeneres-christmas-card/
https://www.kickstarter.com//projects/77420923/thru-hike-the-appalachian-trail-game/
https://www.kickstarter.com//projects/1522559512/48-hour-film-race/
https://www.kickstarter.com//projects/1887486731/1872-steffey-house-restoration-project-saving-hist/
https://www.kickstarter.com//projects/984492639/operator-short-film/
https://www.kickstarter.com//projects/1085318100/red-blue/
https://www.kickstarter.com//projects/572490810/take-off-0/
https://www.kickstarter.com//projects/1357414043/bobbie-beat-down/
https://www.kickstarter.com//projects/818418241/hydration-for-the-nation-get-the-country-healthier/
https://www.kickstarter.com//projects/1712452111/red-boot-travel/
https://www.kickstarter.com//projects/1484064877/spice-r

https://www.kickstarter.com//projects/667392092/slimfold-wallet-soft-shell-70mph-crash-tested-dura/
https://www.kickstarter.com//projects/1068560787/chocolate-peanut-butter-fudge/
https://www.kickstarter.com//projects/1570910354/kimo-baker-cd-project/
https://www.kickstarter.com//projects/835703068/art-and-empowerment-mural/
https://www.kickstarter.com//projects/crazycoolgrandma/crazy-cool-grandmas-comedy-review-iii/
https://www.kickstarter.com//projects/676971916/faygo-huntin-made-simple-with-this-new-app/
https://www.kickstarter.com//projects/134623818/nwczradiocom-needs-you/
https://www.kickstarter.com//projects/506983569/desserts-by-tiffany/
https://www.kickstarter.com//projects/1937253848/fife-and-drum-syw-austrian-artillery/
https://www.kickstarter.com//projects/thebigredflagbook/the-big-red-flag-book-analyzes-common-relationship/
https://www.kickstarter.com//projects/1752896151/snowy-oak-sausage-company/
https://www.kickstarter.com//projects/briandickman/spark-young-engineers-wi

https://www.kickstarter.com//projects/1285658449/moody/
https://www.kickstarter.com//projects/1333622279/banknote-2nd-edition-minimalist-leather-wallet-and/
https://www.kickstarter.com//projects/581482121/nemo-and-giraffe/
https://www.kickstarter.com//projects/1843511375/salad-jar-a-mason-jar-salad-vending-machine/
https://www.kickstarter.com//projects/smartfilmschool/smart-film-school-a-complete-film-school-in-your-p/
https://www.kickstarter.com//projects/466435676/artilect-music-for-humans-by-humans/
https://www.kickstarter.com//projects/gboy/groceryboy-the-revolutionary-new-grocery-bag-carri/
https://www.kickstarter.com//projects/1239046846/your-bowling-caddy/
https://www.kickstarter.com//projects/1363059024/nerds-of-online-podcast-nerdcast/
https://www.kickstarter.com//projects/1987945596/offpiste-the-revolutionary-ultrapack-for-your-adve/
https://www.kickstarter.com//projects/artwithheart/secret-diaries/
https://www.kickstarter.com//projects/1840526642/sexploitation-the-daniel-per

In [40]:
projects.shape

(1995, 11)

In [ ]:
#selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.binary_location = '/usr/bin/google-chrome'

chromedriver = "/bin/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver


driver = webdriver.Chrome("/usr/lib/chromium-browser/chromedriver")


for page_num in page_numbers:
    url = 'https://www.kicktraq.com/archive/?page='+str(page_num)
    driver.get(url)
    time.sleep(10)

In [23]:
projects.shape

(1995, 11)